In [2]:
import pandas as pd
#Import Data
df = pd.read_csv('data.csv', sep=',', header=0)
#clean
df = df.rename(columns=lambda x: x.strip())
df = df.dropna()
df

,ID,GROUPE,MUSCLE,SWE_perc,SWE_raw,SWE_dist,ACTIVATION,LRELATIVE,LRELATIVE_pic,F_V,ForceRelative,ForceRelative_pic,ChargeInterne
0,1,NHE,SM,0.0,-0.1,0.00,0.341907,0.646984,0.996095,1.084533,0.263361,0.440011,0.256139
1,2,NHE,SM,6.0,2.0,0.18,0.602638,0.529459,0.841290,1.096524,0.355184,0.622870,0.368125
2,3,NHE,SM,0.0,-1.4,0.00,0.418067,0.683204,1.028167,1.089420,0.361932,0.722366,0.344897
3,4,NHE,SM,0.0,-7.6,0.00,0.397005,0.567855,0.935770,1.089609,0.277134,0.567802,0.270510
4,5,NHE,SM,0.0,-7.6,0.00,0.464852,0.718161,0.961237,1.064402,0.395749,0.636120,0.377784
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,17,DL,BF,31.1,9.9,0.60,0.116118,1.288176,1.508788,1.094721,0.305250,0.571073,0.162181
59,18,DL,BF,0.0,-2.5,0.00,0.596198,1.130732,1.372417,1.079483,0.631827,0.824287,0.751614
60,19,DL,BF,17.6,6.4,0.66,0.340944,1.209678,1.434567,1.077420,0.430590,0.533246,0.449137
61,20,DL,BF,25.7,8.9,0.43,0.340485,1.302315,1.515599,1.078430,0.512893,0.774861,0.486235
